### Import Packages

In [1]:
import pandas as pd 
import os 
import numpy as np 
import re
import string
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import math
from selenium.common.exceptions import NoSuchElementException
import googlemaps

### Load Webdriver

In [2]:
EXE_PATH = r"C:\Users\nikhi\Downloads\chromedriver_win32\chromedriver.exe"
browser = webdriver.Chrome(executable_path=EXE_PATH)

### Open Delhi Metro Website

In [53]:
base = "https://www.delhimetrorail.com/"
browser.get(base)
browser.implicitly_wait(10)

In [54]:
browser.find_element_by_id("FromStation").click()

In [55]:
html = browser.page_source
soup = BeautifulSoup(html)

### Find all the different lines (routes)

In [108]:
line_results = browser.find_elements_by_class_name('popup-result-line')
lines = soup.find_all('div', class_ = 'popup-result-line')

### Scrape Data for All Stations on all the lines

In [115]:
stations_prev = ''
dic = {}
count = 0
for i in range(len(line_results)-1):
    
    line_name = lines[i].text
    
    line_results[i].click()

    while True:
        html = browser.page_source
        soup = BeautifulSoup(html)
        if str(soup.find('div', class_ = 'sub-popup-result').find('div', class_ = 'layout')) != stations_prev:
            break
            
    print(line_name)
    stations = soup.find('div', class_ = 'sub-popup-result').find('div', class_ = 'layout').find_all('a', class_ = 'clearfix')
    
    for station in stations:
        dic[count] = {"Line": line_name}
        dic[count]['n'] = station.find('div', class_ = 'sub-result-left').text
        dic[count]['name'] = station.find('div', class_ = 'sub-result-name').text
        dic[count]['disabled friendly'] = 1 if "Divyang" in str(station.find('div', class_ = 'sub-result-list')) else 0
        dic[count]['parking available'] = 1 if "Parking" in str(station.find('div', class_ = 'sub-result-list')) else 0
        dic[count]['elevator available'] = 1 if "Lift" in str(station.find('div', class_ = 'sub-result-list')) else 0
        count+=1
        
    stations_prev = str(soup.find('div', class_ = 'sub-popup-result').find('div', class_ = 'layout'))

Red Line 1
Yellow Line 2
Blue Line 3
Blue Line 4
Green Line 5
Violet Line 6
Pink Line 7
Magenta Line 8
Grey Line 9
Orange Line 10
RMGL 11


In [117]:
pd.DataFrame.from_dict(dic).T

,Line,n,name,disabled friendly,parking available,elevator available
0,Red Line 1,1,SHAHEED STHAL ( NEW BUS ADDA),1,1,1
1,Red Line 1,2,HINDON RIVER,1,1,1
2,Red Line 1,3,ARTHALA,1,0,1
3,Red Line 1,4,MOHAN NAGAR,1,0,1
4,Red Line 1,5,SHYAM PARK,1,0,1
...,...,...,...,...,...,...
260,RMGL 11,7,PHASE 2,1,0,1
261,RMGL 11,8,BELVEDERE TOWERS,1,0,1
262,RMGL 11,9,CYBER CITY,1,0,1
263,RMGL 11,10,MOULSARI AVENUE,1,0,1


### Geocode Stations using Google Maps API

In [140]:
g_API = 'api_key'
gmaps_key = googlemaps.Client(key=g_API)

for i in range(len(dic)):
    search_term = "Delhi Metro " + dic[i]['Line'][:-2] + " " + dic[i]['name']
    geocode_obj = gmaps_key.geocode(search_term)
    dic[i]['map_info'] = geocode_obj

In [145]:
for i in range(len(dic)):
    if len(dic[i]['map_info']) > 0:
        lat = dic[i]['map_info'][0]['geometry']['location']['lat']
        lon = dic[i]['map_info'][0]['geometry']['location']['lng']
        dic[i]['lat'] = lat
        dic[i]['long'] = lon
        
    else:
        print(dic[i])

{'Line': 'Violet Line 6', 'n': '28', 'name': 'BADKAL MOR', 'disabled friendly': 1, 'parking available': 0, 'elevator available': 1, 'map_info': []}
{'Line': 'Pink Line 7', 'n': '25', 'name': 'EAST VINOD NAGAR-MAYUR VIHAR -II', 'disabled friendly': 1, 'parking available': 0, 'elevator available': 1, 'map_info': []}
{'Line': 'Magenta Line 8', 'n': '11', 'name': 'I.I.T', 'disabled friendly': 1, 'parking available': 0, 'elevator available': 1, 'map_info': []}
{'Line': 'RMGL 11', 'n': '5', 'name': 'PHASE 1', 'disabled friendly': 1, 'parking available': 0, 'elevator available': 1, 'map_info': []}
{'Line': 'RMGL 11', 'n': '11', 'name': 'PHASE 3', 'disabled friendly': 1, 'parking available': 0, 'elevator available': 1, 'map_info': []}


In [151]:
df = pd.DataFrame.from_dict(dic).T

In [155]:
df.drop(columns=['map_info']).to_excel(r"C:\Users\nikhi\Downloads\Delhi_metro.xlsx")  